# Find all must support elements in StructureDefinitions

- for hard coded path search all SD (in YAML) for must support = true elements
- list em in a md table format in std out
- update em with with teh USCDI extension and write back to YAML

In [28]:
from pathlib import Path
from json import load, dumps
from yaml import load as y_load, dump as y_dump, FullLoader
from fhir.resources import construct_fhir_element
from pprint import pprint
my_path = Path('/Users/ehaas/Documents/FHIR/US-Core/input/examples-yaml')
# snapshot_path =  Path('/Users/ehaas/Documents/FHIR/US-Core/output')
my_path

PosixPath('/Users/ehaas/Documents/FHIR/US-Core/input/examples-yaml')

In [29]:
#find  where to insert new element
# search recursively for a value = test and return the path to the value
# add an element as a sibling to the path


def find_and_insert(my_dict, path, new_element):
    test = 'http://snomed.info/sct'
    new_key = 'version'
    new_element = "http://snomed.info/sct/731000124108"
    for k, v in my_dict.items():
        path.append(k)
        if isinstance(v, dict):
            find_and_insert(v, path, new_element)
        elif isinstance(v, list):
            for i, item in enumerate(v):
                if isinstance(item, dict):
                    path.append(i)
                    find_and_insert(item, path, new_element)
                    path.pop()
        elif v == test:
            print('Found test at:', path)
            #insert new element
            my_dict[new_key] = new_element
            print(f'inserted new_element at {path}')
            return
        path.pop()
        # print(y_dump(my_dict,indent=2,sort_keys=False))


for enum,i in enumerate(my_path.glob('allergy*.yml')):
    # print(i.read_text())
    obj = y_load(i.read_text(),Loader=FullLoader) #dict
    # if obj['id'] not in []:
    #     continue
    print(enum,i,obj['resourceType'])
    print(obj['id'])
    find_and_insert(obj, [], 'new_element')
    print('-----------------')
    print(f'writing to {i}...')
    i.write_text(y_dump(obj,indent=2,sort_keys=False))





0 /Users/ehaas/Documents/FHIR/US-Core/input/examples-yaml/allergyintolerance-provenance-example.yml Bundle
c887e62f-6166-419f-8268-b5ecd6c7b901
Found test at: ['entry', 0, 'resource', 'code', 'coding', 0, 'system']
inserted new_element at ['entry', 0, 'resource', 'code', 'coding', 0, 'system']
Found test at: ['entry', 0, 'resource', 'code', 'reaction', 0, 'manifestation', 0, 'coding', 0, 'system']
inserted new_element at ['entry', 0, 'resource', 'code', 'reaction', 0, 'manifestation', 0, 'coding', 0, 'system']
-----------------
writing to /Users/ehaas/Documents/FHIR/US-Core/input/examples-yaml/allergyintolerance-provenance-example.yml...
1 /Users/ehaas/Documents/FHIR/US-Core/input/examples-yaml/allergyintolerance-example.yml AllergyIntolerance
example
Found test at: ['code', 'coding', 0, 'system']
inserted new_element at ['code', 'coding', 0, 'system']
Found test at: ['code', 'reaction', 0, 'manifestation', 0, 'coding', 0, 'system']
inserted new_element at ['code', 'reaction', 0, 'mani

In [30]:



    for e in obj:
        try:
            if  e['id'] in uscdi_whitelist or e['mustSupport']:
                count += 1
                # print(f"e['id'] = {e['id']}")
                # add extension to the element
                # print(dumps(e, indent=2))
                try:
                    e['extension'].append(my_ext)
                except KeyError:
                    e['extension'] = [my_ext]
                # need to add to the short from the snapshot
                #get snapshot_short
                try:
                    snapshot_short = (s for s in snapshot['snapshot']['element'] if s['id'] == e['id']).__next__()['short']
                except StopIteration:
                    try:
                        snapshot_short = (s for s in snapshot['snapshot']['element'] if s['id'] == ms_id_map[e['id']]).__next__()['short']
                    except StopIteration:
                        cant_match_path.append((e['id'],e['path']))
                print(".", end='')
                # print(snapshot_short)
                try:
                    e['short'] = f"(USCDI Requirement) {snapshot_short}"
                except KeyError:
                    e['short'] = '(USCDI Requirement)'

                # print(yaml.dump(e, indent=2))
                # list the choices
                # for snapshot in obj['snapshot']['element']:
                #     if e['path'] == snapshot['path']:
                #     # print(snapshot['path'])
                #         types = [type['code'] for type in snapshot['type'] if type['code'] in 
                #                  ['code',"Coding","CodeableConcept"]]
                #         if types and snapshot['binding']['strength']=='extensible':
                #             print(f"{obj['title']}|{e['path']}|e min: {e['min']}|{types}|{snapshot['binding']['strength']}")
                #         break
                # print( f'writing file to {i}')
                # fhir_obj = construct_fhir_element(obj['resourceType'], obj) # sort elements and validate
                # # i.write_text(fhir_obj.yaml(indent=True))

                # print(fhir_obj.yaml(indent=True))
                

        except KeyError:
            pass
    print()
    print( f"writing {obj['id']} to {i}\n\n")
    fhir_obj = construct_fhir_element(obj['resourceType'], obj) # sort elements and validate
    i.write_text(fhir_obj.yaml(indent=True))

    # print(fhir_obj.yaml(indent=True))
    # if count > 25 : break
print(f'{enum} profiles with {count} mustSupport elements added to the differential')

print(f'cant_match_path = {set(cant_match_path)}')

TypeError: string indices must be integers